In [1]:

import sys
sys.path.append('/home/techt/Desktop/a4s')


### Step 1: Initial Data Loading

In [2]:
from utils.preprocess import *
# 1. Identify the paths to the soft file and the matrix file
cohort_dir = '/media/techt/DATA/GEO/Ocular_Melanomas/GSE78033'
soft_file, matrix_file = geo_get_relevant_filepaths(cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Expression Data from Uveal Melanoma patient-derived xenograft and tumor of origin"
!Series_summary	"We compare the genetic profiles of the primary tumors of uveal melanoma or metastasis to their corresponding xenografts that have been passaged over time."
!Series_summary	"The study included patient tumors and corresponding xenografts at very early (P1), early (P4), and late (P9) in vivo passages"
!Series_overall_design	"The transcriptome of 16 uveal melanoma patient-derived xenografts models were analyzed using Affymetrix Human Exon 1.0 ST Array"
Sample Characteristics Dictionary:
{0: ['sample type: PDX', 'sample type: Patient'], 1: ['sample origin: Xenograft', 'sample origin: liver metastasis', 'sample origin: primary ocular tumor', 'sample origin: skin metastasis'], 2: ['models: MM026', 'models: MM033', 'models: MP042', 'models: MM066', 'models: MP077', 'models: MP041', 'models: MP034', 'models: MP047', 'models: MP071', 'models: MP080', 'models:

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
is_gene_available = False
trait_row = age_row = gender_row = None  # set to different values when applicable
convert_trait = convert_age = convert_gender = None  # define the functions when applicable

# Determine if gene expression data is available
is_gene_available = True  # Based on !Series_overall_design mentioning Affymetrix Human Exon 1.0 ST Array

# Evaluate data availability for each variable
if any("primary ocular tumor" in s for s in {1: ['sample origin: Xenograft', 'sample origin: liver metastasis', 'sample origin: primary ocular tumor', 'sample origin: skin metastasis']}[1]):
    trait_row = 1

age_row = None  # No age-related information found in the dictionary
gender_row = None  # No gender-related information found in the dictionary

# Define conversion functions
def convert_trait(value):
    if 'primary ocular tumor' in value:
        return 1
    elif 'liver metastasis' in value or 'skin metastasis' in value:
        return 0
    else:
        return None  # unknown or irrelevant cases

def convert_age(value):
    return None  # No age data available

def convert_gender(value):
    return None  # No gender data available

# Save cohort information
save_cohort_info('GSE78033', './preprocessed/Ocular_Melanomas/cohort_info.json', is_gene_available, trait_row is not None)

# Clinical Feature Extraction
if trait_row is not None:
    selected_clinical_data = geo_select_clinical_features(clinical_data, 'Ocular_Melanomas', trait_row, convert_trait, age_row, convert_age, gender_row, convert_gender)
    csv_path = './preprocessed/Ocular_Melanomas/trait_data/GSE78033.csv'
    selected_clinical_data.to_csv(csv_path)
    print(preview_df(selected_clinical_data))


{'GSM2065182': [None], 'GSM2065183': [None], 'GSM2065184': [None], 'GSM2065185': [None], 'GSM2065186': [None], 'GSM2065187': [None], 'GSM2065188': [None], 'GSM2065189': [0], 'GSM2065190': [1], 'GSM2065191': [1], 'GSM2065192': [1], 'GSM2065193': [None], 'GSM2065194': [None], 'GSM2065195': [None], 'GSM2065196': [None], 'GSM2065197': [None], 'GSM2065198': [None], 'GSM2065199': [None], 'GSM2065200': [None], 'GSM2065201': [None], 'GSM2065202': [None], 'GSM2065203': [None], 'GSM2065204': [None], 'GSM2065205': [None], 'GSM2065206': [0], 'GSM2065207': [1], 'GSM2065208': [1], 'GSM2065209': [0], 'GSM2065210': [1], 'GSM2065211': [None], 'GSM2065212': [None], 'GSM2065213': [None], 'GSM2065214': [None], 'GSM2065215': [None], 'GSM2065216': [None], 'GSM2065217': [None], 'GSM2065218': [None], 'GSM2065219': [None], 'GSM2065220': [None], 'GSM2065221': [None], 'GSM2065222': [None], 'GSM2065223': [1], 'GSM2065224': [0], 'GSM2065225': [1], 'GSM2065226': [None]}


### Step 3: Gene Data Extraction

In [4]:
# 1. Use the get_genetic_data function from the library to get the gene_data from the matrix_file previously defined.
gene_data = get_genetic_data(matrix_file)

# 2. Print the first 20 row ids for the following step.
print(gene_data.index[:20])


Index(['100008589_at', '100009676_at', '10000_at', '10001_at', '10002_at',
       '100033416_at', '100033423_at', '100033424_at', '100033425_at',
       '100033426_at', '100033428_at', '100033431_at', '100033436_at',
       '100033444_at', '100033453_at', '100033806_at', '100033820_at',
       '100037417_at', '100038246_at', '10003_at'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
requires_gene_mapping = True


### Step 5: Gene Annotation (Conditional)

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the soft file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


Gene annotation preview:
{'ID': ['1_at', '10_at', '100_at', '1000_at', '10000_at'], 'ENTREZ_GENE_ID': ['1', '10', '100', '1000', '10000'], 'SPOT_ID': ['A1BG', 'NAT2', 'ADA', 'CDH2', 'AKT3']}


### Step 6: Gene Identifier Mapping

In [7]:
# 1. Identifying the key names based on the dictionary preview
identifier_key = 'ID'
gene_symbol_key = 'SPOT_ID'

# 2. Get the dataframe storing the mapping between probe IDs and genes
gene_mapping = get_gene_mapping(gene_annotation, identifier_key, gene_symbol_key)

# 3. Apply the mapping to get the gene expression dataframe
gene_data = apply_gene_mapping(gene_data, gene_mapping)


### Step 7: Data Normalization and Merging

In [8]:
import os

# Ensure directories exist
gene_csv_path = './preprocessed/Ocular_Melanomas/gene_data/GSE78033.csv'
csv_path = './preprocessed/Ocular_Melanomas/GSE78033.csv'
os.makedirs(os.path.dirname(gene_csv_path), exist_ok=True)
os.makedirs(os.path.dirname(csv_path), exist_ok=True)

# 1. Normalize the obtained gene data with the 'normalize_gene_symbols_in_index' function from the library.
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
normalized_gene_data.to_csv(gene_csv_path)

# 2. Merge the clinical and genetic data with the 'geo_merge_clinical_genetic_data' function from the library.
merged_data = geo_merge_clinical_genetic_data(selected_clinical_data, normalized_gene_data)

# 3. Determine whether the trait and some demographic attributes in the data are severely biased, and remove biased attributes.
trait_biased, unbiased_merged_data = judge_and_remove_biased_features(merged_data, 'Ocular_Melanomas')

# If the trait is not severely biased, save the cohort information and the merged data.

# 4. Save the cohort information.
save_cohort_info('GSE78033', './preprocessed/Ocular_Melanomas/cohort_info.json', True, True, trait_biased, merged_data)

if not trait_biased:
    # 5. If the trait is not severely biased, save the merged data to a csv file.
    unbiased_merged_data.to_csv(csv_path)


For the feature 'Ocular_Melanomas', the least common label is '0.0' with 4 occurrences. This represents 33.33% of the dataset.
The distribution of the feature 'Ocular_Melanomas' in this dataset is fine.

A new JSON file was created at: ./preprocessed/Ocular_Melanomas/cohort_info.json
